# Exploração de dados

## Importar dados de reclamação

In [240]:
import pandas as pd
import numpy as np

In [241]:
caminho = '/content/drive/MyDrive/0. STF/01_Projetos/2023/07.05_reclamacoes/reclamacao_fev_jun.gzip'
df = pd.read_parquet(caminho)

In [242]:
df.head()

,Link Processo,Número,Relator Atual,Segredo Justiça,Situação Processo,Alertas,Data da Autuação,Partes - Polos Ativos,Partes - Polos Passivos,Indicador Usurpação,Usurpação de Competência,Repercussão Geral,Controle Concentrado,Sumula,Controle Difuso,Outros
0,Rcl-57762,57762,MIN. CÁRMEN LÚCIA,Não,Distribuido,Medida Liminar Criminal,2023-02-01,JEFERSON LUIZ DE SANTANA,JUIZ DE DIREITO DA VARA ÚNICA DA COMARCA DE CA...,Violação,nan,TEMA 608,nan,nan,nan,nan
1,Rcl-57763,57763,MIN. NUNES MARQUES,Não,Distribuido,Medida Liminar,2023-02-01,JOSE FERREIRA CUNHA,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO,Usurpação,nan,nan,nan,nan,nan,nan
2,Rcl-57764,57764,MIN. LUIZ FUX,Não,Distribuido,nan,2023-02-01,ESTADO DE MATO GROSSO DO SUL,TRIBUNAL DE JUSTICA DO ESTADO DE MATO GROSSO D...,Violação,nan,TEMA 793,nan,nan,nan,nan
3,Rcl-57765,57765,MIN. LUIZ FUX,Não,Distribuido,nan,2023-02-01,ESTADO DE MATO GROSSO DO SUL,TRIBUNAL DE JUSTICA DO ESTADO DE MATO GROSSO D...,Violação,nan,TEMA 793,nan,nan,nan,nan
4,Rcl-57766,57766,MIN. EDSON FACHIN,Não,Distribuido,nan,2023-02-01,VERA LUCIA GONCALVES DOS SANTOS,QUARTA TURMA RECURSAL DOS JUIZADOS ESPECIAIS D...,Violação,nan,nan,nan,nan,MI 7.044,nan


In [243]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2966 entries, 0 to 600
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Link Processo             2966 non-null   object        
 1   Número                    2966 non-null   object        
 2   Relator Atual             2966 non-null   object        
 3   Segredo Justiça           2966 non-null   object        
 4   Situação Processo         2966 non-null   object        
 5   Alertas                   2966 non-null   object        
 6   Data da Autuação          2966 non-null   datetime64[ns]
 7   Partes - Polos Ativos     2966 non-null   object        
 8   Partes - Polos Passivos   2966 non-null   object        
 9   Indicador Usurpação       2966 non-null   object        
 10  Usurpação de Competência  2966 non-null   object        
 11  Repercussão Geral         2966 non-null   object        
 12  Controle Concentrado 

In [244]:
import datetime
from datetime import date, timedelta

df['Data da Autuação'].dt.strftime('%Y.%m').value_counts()

2023.03    697
2023.05    653
2023.04    622
2023.06    601
2023.02    393
Name: Data da Autuação, dtype: int64

In [245]:
# Quantidade de dados nulos
pd.isnull(df['Indicador Usurpação']).value_counts()

False    2966
Name: Indicador Usurpação, dtype: int64

In [246]:
df['Indicador Usurpação'].isna().sum()

0

In [247]:
# df[df.isna().any(axis=1)]
# df.loc[df.isnull().any(axis=1)]

## Importar dados de decisão

In [248]:
# importa dados de decisão
caminho = '/content/drive/MyDrive/0. STF/01_Projetos/2023/07.05_reclamacoes/decisoes_rcl_ate_hoje.xlsx'
df_decisao = pd.read_excel(caminho)

In [249]:
df_decisao.shape

(95641, 17)

In [250]:
df_decisao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95641 entries, 0 to 95640
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Classe                      95641 non-null  object        
 1   Número                      95641 non-null  int64         
 2   Link                        95641 non-null  object        
 3   Relator                     95641 non-null  object        
 4   Data de autuação            95641 non-null  datetime64[ns]
 5   Indicador virtual           95641 non-null  object        
 6   Data da decisão             95641 non-null  datetime64[ns]
 7   Ano da decisão              95641 non-null  int64         
 8   Subgrupo andamento decisão  95641 non-null  object        
 9   Andamento decisão           95641 non-null  object        
 10  Observação do andamento     95641 non-null  object        
 11  Indicador Colegiada         95641 non-null  object    

In [251]:
# Create new variable
df_decisao['Link Processo'] = df_decisao['Classe'] + '-' + df_decisao['Número'].astype(str)

## Retirar decisãos repitadas

In [252]:
# Filtrar os processos com mais de duas decisões

filtered_decision = df_decisao[df_decisao['Link Processo'].isin(df['Link Processo'])]
decision_rcl = pd.DataFrame(filtered_decision['Link Processo'].value_counts())

condition_filter = decision_rcl[decision_rcl['Link Processo'] > 1]
condition_filter

,Link Processo
Rcl-58301,6
Rcl-57957,6
Rcl-58292,5
Rcl-58587,5
Rcl-58919,5
...,...
Rcl-57951,2
Rcl-57917,2
Rcl-57874,2
Rcl-60345,2


In [253]:
# size of database
filtered_decision.shape

(3231, 18)

In [254]:
# Vamos retirar os processos acima que tem mais de uma decisão e não é decisão final
drop_count2 = df_decisao[(df_decisao['Link Processo'].isin(condition_filter.index)) &
                         (df_decisao['Subgrupo andamento decisão'] != 'Decisão Final')]

df_decisao.drop(drop_count2.index, inplace = True)
df_decisao.shape

(95012, 18)

In [255]:
# Filtrar os processos que persistem com mais de duas decisões

filtered_decision_pos = df_decisao[df_decisao['Link Processo'].isin(df['Link Processo'])]
decision_rcl_pos = pd.DataFrame(filtered_decision_pos['Link Processo'].value_counts())

condition_filter_pos = decision_rcl_pos[decision_rcl_pos['Link Processo'] > 1]
condition_filter_pos

,Link Processo
Rcl-57957,4
Rcl-58775,3
Rcl-58255,2
Rcl-58059,2
Rcl-60630,2
Rcl-58549,2
Rcl-60354,2
Rcl-59959,2
Rcl-60137,2
Rcl-57762,2


## Juntar as bases

In [256]:
# df['Link Processo'].value_counts().sort_values()

In [257]:
# selecao das variáveis importantes
selecao_variaveis = ['Link Processo', 'Classe', 'Data da decisão', 'Andamento decisão',
                     'Observação do andamento', 'Indicador Colegiada', 'Ramo direito', 'Assuntos do processo'
                    ]

df_decisao_to_merge = df_decisao[selecao_variaveis]

In [258]:
# dados de decisão em reclamações
df_decisao_to_merge.shape

(95012, 8)

In [259]:
# dados de reclamações recebidas
df.shape

(2966, 16)

In [260]:
# Join das bases em que as bases de recebimento são a referência

df_total = pd.merge(df, df_decisao_to_merge, how='left', on='Link Processo')
df_total.shape

(3027, 23)

In [261]:
df_total['Link Processo'].value_counts().sort_values(ascending = False)

Rcl-57957    4
Rcl-58775    3
Rcl-58697    2
Rcl-59959    2
Rcl-58562    2
            ..
Rcl-60692    1
Rcl-60693    1
Rcl-60694    1
Rcl-60695    1
Rcl-60731    1
Name: Link Processo, Length: 2966, dtype: int64

In [262]:
# Selecionamos as linhas que devem ser dropadas
df_total_big2 = df_total[df_total['Link Processo'].map(df_total['Link Processo'].value_counts()) > 1]

selecao_to_drop =  ['Link Processo', 'Classe', 'Número']
index_to_drop = df_total_big2[selecao_to_drop].drop_duplicates(keep = 'last')

index_to_drop

,Link Processo,Classe,Número
1,Rcl-57762,Rcl,57762
109,Rcl-57869,Rcl,57869
166,Rcl-57926,Rcl,57926
175,Rcl-57934,Rcl,57934
178,Rcl-57936,Rcl,57936
202,Rcl-57957,Rcl,57957
239,Rcl-57993,Rcl,57993
247,Rcl-58000,Rcl,58000
307,Rcl-58059,Rcl,58059
468,Rcl-58219,Rcl,58219


In [263]:
# df_total retirando os duplicados
df_total.drop(index_to_drop.index, inplace = True)

In [264]:
df_total['Link Processo'].value_counts()

Rcl-57957    3
Rcl-58775    2
Rcl-57762    1
Rcl-59745    1
Rcl-59736    1
            ..
Rcl-58753    1
Rcl-58754    1
Rcl-58755    1
Rcl-58756    1
Rcl-60731    1
Name: Link Processo, Length: 2966, dtype: int64

In [265]:
selecao = df_total['Link Processo'] == 'Rcl-57957'
retirar = df_total[selecao]
# id = [198, 200]
# retirar.iloc[]

index_drop = retirar.iloc[[0,2]]
df_total.drop(index_drop.index, inplace = True)

In [266]:
df_total.shape

(2967, 23)

## Criar uma variável de provido

In [267]:
# Listar os andamentos
andamentos = df_total['Andamento decisão'].value_counts()
andamentos = pd.DataFrame(andamentos).reset_index().iloc[:,[0]]
andamentos.rename(columns = {'index' : 'Andamento decisão'}, inplace = True)

andamentos

,Andamento decisão
0,Negado seguimento
1,Procedente
2,Procedente em parte
3,Improcedente
4,Prejudicado
5,Extinto o processo
6,Homologada a desistência
7,Liminar deferida
8,Decisão (segredo de justiça)
9,Não conhecido(s)


In [268]:
lista_provimento = [
  'Agravo provido e desde logo provido o RE',
  'Agravo provido e desde logo provido parcialmente o RE',
  'Conhecido e provido',
  'Conhecido e provido em parte',
  'Conhecido em parte e nessa parte provido',
  'Conhecido em parte e nessa parte parcialmente provido',
  'Procedente',
  'Procedente em parte',
  'Provido',
  'Provido em parte',
  'Concedida a ordem de ofício'
]

# transformar a lista em DF
df_provimento = pd.DataFrame(lista_provimento, columns =  ['Andamento decisão'])
df_provimento['Provimento'] = 'provido'
df_provimento

,Andamento decisão,Provimento
0,Agravo provido e desde logo provido o RE,provido
1,Agravo provido e desde logo provido parcialmen...,provido
2,Conhecido e provido,provido
3,Conhecido e provido em parte,provido
4,Conhecido em parte e nessa parte provido,provido
5,Conhecido em parte e nessa parte parcialmente ...,provido
6,Procedente,provido
7,Procedente em parte,provido
8,Provido,provido
9,Provido em parte,provido


In [269]:
andamentos = andamentos.merge(df_provimento, on = 'Andamento decisão', how = 'left')
andamentos['Provimento'].fillna('não provido', inplace = True)
andamentos

# 14 -> provido
# 15 -> provido
# 9 -> deferida

,Andamento decisão,Provimento
0,Negado seguimento,não provido
1,Procedente,provido
2,Procedente em parte,provido
3,Improcedente,não provido
4,Prejudicado,não provido
5,Extinto o processo,não provido
6,Homologada a desistência,não provido
7,Liminar deferida,não provido
8,Decisão (segredo de justiça),não provido
9,Não conhecido(s),não provido


In [270]:
# Incluindo variável PROVIMENTO no DF
df_total= df_total.merge(df_provimento, on = 'Andamento decisão', how = 'left')

df_total['Provimento'].fillna('não provido', inplace = True)
df_total.head()

,Link Processo,Número,Relator Atual,Segredo Justiça,Situação Processo,Alertas,Data da Autuação,Partes - Polos Ativos,Partes - Polos Passivos,Indicador Usurpação,...,Controle Difuso,Outros,Classe,Data da decisão,Andamento decisão,Observação do andamento,Indicador Colegiada,Ramo direito,Assuntos do processo,Provimento
0,Rcl-57762,57762,MIN. CÁRMEN LÚCIA,Não,Distribuido,Medida Liminar Criminal,2023-02-01,JEFERSON LUIZ DE SANTANA,JUIZ DE DIREITO DA VARA ÚNICA DA COMARCA DE CA...,Violação,...,nan,nan,Rcl,2023-02-03,Negado seguimento,"""[...] 9. Pelo exposto, nego seguimento à pres...",MONOCRÁTICA,DIREITO PROCESSUAL PENAL | PRISÃO PREVENTIVA |...,1 - DIREITO PROCESSUAL PENAL || PRISÃO PREVENT...,não provido
1,Rcl-57763,57763,MIN. NUNES MARQUES,Não,Distribuido,Medida Liminar,2023-02-01,JOSE FERREIRA CUNHA,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO,Usurpação,...,nan,nan,Rcl,2023-02-09,Negado seguimento,-,MONOCRÁTICA,DIREITO PROCESSUAL CIVIL E DO TRABALHO | FORMA...,1 - DIREITO PROCESSUAL CIVIL E DO TRABALHO || ...,não provido
2,Rcl-57764,57764,MIN. LUIZ FUX,Não,Distribuido,nan,2023-02-01,ESTADO DE MATO GROSSO DO SUL,TRIBUNAL DE JUSTICA DO ESTADO DE MATO GROSSO D...,Violação,...,nan,nan,Rcl,2023-03-30,Improcedente,-,MONOCRÁTICA,DIREITO DA SAÚDE | PÚBLICA | TRATAMENTO MÉDICO...,1 - DIREITO DA SAÚDE || PÚBLICA || TRATAMENTO ...,não provido
3,Rcl-57765,57765,MIN. LUIZ FUX,Não,Distribuido,nan,2023-02-01,ESTADO DE MATO GROSSO DO SUL,TRIBUNAL DE JUSTICA DO ESTADO DE MATO GROSSO D...,Violação,...,nan,nan,Rcl,2023-03-10,Improcedente,"(...) ""Ex positis, JULGO IMPROCEDENTE a reclam...",MONOCRÁTICA,DIREITO DA SAÚDE | PÚBLICA | TRATAMENTO MÉDICO...,1 - DIREITO DA SAÚDE || PÚBLICA || TRATAMENTO ...,não provido
4,Rcl-57766,57766,MIN. EDSON FACHIN,Não,Distribuido,nan,2023-02-01,VERA LUCIA GONCALVES DOS SANTOS,QUARTA TURMA RECURSAL DOS JUIZADOS ESPECIAIS D...,Violação,...,MI 7.044,nan,Rcl,2023-05-31,Negado seguimento,-,MONOCRÁTICA,DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...,1 - DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS D...,não provido


## Limpeza final da base

In [271]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2967 entries, 0 to 2966
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Link Processo             2967 non-null   object        
 1   Número                    2967 non-null   object        
 2   Relator Atual             2967 non-null   object        
 3   Segredo Justiça           2967 non-null   object        
 4   Situação Processo         2967 non-null   object        
 5   Alertas                   2967 non-null   object        
 6   Data da Autuação          2967 non-null   datetime64[ns]
 7   Partes - Polos Ativos     2967 non-null   object        
 8   Partes - Polos Passivos   2967 non-null   object        
 9   Indicador Usurpação       2967 non-null   object        
 10  Usurpação de Competência  2967 non-null   object        
 11  Repercussão Geral         2967 non-null   object        
 12  Controle Concentrado

In [272]:
# Verificar se há alguma coluna duplicada
df_total.columns.duplicated().sum()

0

In [273]:
df_total.head()

,Link Processo,Número,Relator Atual,Segredo Justiça,Situação Processo,Alertas,Data da Autuação,Partes - Polos Ativos,Partes - Polos Passivos,Indicador Usurpação,...,Controle Difuso,Outros,Classe,Data da decisão,Andamento decisão,Observação do andamento,Indicador Colegiada,Ramo direito,Assuntos do processo,Provimento
0,Rcl-57762,57762,MIN. CÁRMEN LÚCIA,Não,Distribuido,Medida Liminar Criminal,2023-02-01,JEFERSON LUIZ DE SANTANA,JUIZ DE DIREITO DA VARA ÚNICA DA COMARCA DE CA...,Violação,...,nan,nan,Rcl,2023-02-03,Negado seguimento,"""[...] 9. Pelo exposto, nego seguimento à pres...",MONOCRÁTICA,DIREITO PROCESSUAL PENAL | PRISÃO PREVENTIVA |...,1 - DIREITO PROCESSUAL PENAL || PRISÃO PREVENT...,não provido
1,Rcl-57763,57763,MIN. NUNES MARQUES,Não,Distribuido,Medida Liminar,2023-02-01,JOSE FERREIRA CUNHA,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO,Usurpação,...,nan,nan,Rcl,2023-02-09,Negado seguimento,-,MONOCRÁTICA,DIREITO PROCESSUAL CIVIL E DO TRABALHO | FORMA...,1 - DIREITO PROCESSUAL CIVIL E DO TRABALHO || ...,não provido
2,Rcl-57764,57764,MIN. LUIZ FUX,Não,Distribuido,nan,2023-02-01,ESTADO DE MATO GROSSO DO SUL,TRIBUNAL DE JUSTICA DO ESTADO DE MATO GROSSO D...,Violação,...,nan,nan,Rcl,2023-03-30,Improcedente,-,MONOCRÁTICA,DIREITO DA SAÚDE | PÚBLICA | TRATAMENTO MÉDICO...,1 - DIREITO DA SAÚDE || PÚBLICA || TRATAMENTO ...,não provido
3,Rcl-57765,57765,MIN. LUIZ FUX,Não,Distribuido,nan,2023-02-01,ESTADO DE MATO GROSSO DO SUL,TRIBUNAL DE JUSTICA DO ESTADO DE MATO GROSSO D...,Violação,...,nan,nan,Rcl,2023-03-10,Improcedente,"(...) ""Ex positis, JULGO IMPROCEDENTE a reclam...",MONOCRÁTICA,DIREITO DA SAÚDE | PÚBLICA | TRATAMENTO MÉDICO...,1 - DIREITO DA SAÚDE || PÚBLICA || TRATAMENTO ...,não provido
4,Rcl-57766,57766,MIN. EDSON FACHIN,Não,Distribuido,nan,2023-02-01,VERA LUCIA GONCALVES DOS SANTOS,QUARTA TURMA RECURSAL DOS JUIZADOS ESPECIAIS D...,Violação,...,MI 7.044,nan,Rcl,2023-05-31,Negado seguimento,-,MONOCRÁTICA,DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...,1 - DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS D...,não provido


In [274]:
# selecao das variáveis importantes
selecao_variaveis = ['Link Processo', 'Classe', 'Número', 'Data da Autuação', 'Relator Atual',
                     'Situação Processo', 'Partes - Polos Ativos', 'Partes - Polos Passivos', 'Indicador Usurpação',
                     'Usurpação de Competência','Repercussão Geral', 'Controle Concentrado',
                     'Sumula', 'Controle Difuso', 'Outros',
                     'Data da decisão', 'Andamento decisão', 'Observação do andamento','Provimento',
                     'Indicador Colegiada', 'Ramo direito', 'Assuntos do processo'
                    ]

In [275]:
# selecao de variáveis
df_total = df_total[selecao_variaveis]


In [276]:
import datetime
from datetime import date, timedelta

df_total['ano_autu'] = df_total['Data da Autuação'].dt.strftime('%Y')
df_total['mes_autu'] = df_total['Data da Autuação'].dt.strftime('%m')
df_total['ano_mes_autu'] = df_total['Data da Autuação'].dt.strftime('%Y.%m')

In [277]:
df_total.head()

,Link Processo,Classe,Número,Data da Autuação,Relator Atual,Situação Processo,Partes - Polos Ativos,Partes - Polos Passivos,Indicador Usurpação,Usurpação de Competência,...,Data da decisão,Andamento decisão,Observação do andamento,Provimento,Indicador Colegiada,Ramo direito,Assuntos do processo,ano_autu,mes_autu,ano_mes_autu
0,Rcl-57762,Rcl,57762,2023-02-01,MIN. CÁRMEN LÚCIA,Distribuido,JEFERSON LUIZ DE SANTANA,JUIZ DE DIREITO DA VARA ÚNICA DA COMARCA DE CA...,Violação,nan,...,2023-02-03,Negado seguimento,"""[...] 9. Pelo exposto, nego seguimento à pres...",não provido,MONOCRÁTICA,DIREITO PROCESSUAL PENAL | PRISÃO PREVENTIVA |...,1 - DIREITO PROCESSUAL PENAL || PRISÃO PREVENT...,2023,02,2023.02
1,Rcl-57763,Rcl,57763,2023-02-01,MIN. NUNES MARQUES,Distribuido,JOSE FERREIRA CUNHA,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO,Usurpação,nan,...,2023-02-09,Negado seguimento,-,não provido,MONOCRÁTICA,DIREITO PROCESSUAL CIVIL E DO TRABALHO | FORMA...,1 - DIREITO PROCESSUAL CIVIL E DO TRABALHO || ...,2023,02,2023.02
2,Rcl-57764,Rcl,57764,2023-02-01,MIN. LUIZ FUX,Distribuido,ESTADO DE MATO GROSSO DO SUL,TRIBUNAL DE JUSTICA DO ESTADO DE MATO GROSSO D...,Violação,nan,...,2023-03-30,Improcedente,-,não provido,MONOCRÁTICA,DIREITO DA SAÚDE | PÚBLICA | TRATAMENTO MÉDICO...,1 - DIREITO DA SAÚDE || PÚBLICA || TRATAMENTO ...,2023,02,2023.02
3,Rcl-57765,Rcl,57765,2023-02-01,MIN. LUIZ FUX,Distribuido,ESTADO DE MATO GROSSO DO SUL,TRIBUNAL DE JUSTICA DO ESTADO DE MATO GROSSO D...,Violação,nan,...,2023-03-10,Improcedente,"(...) ""Ex positis, JULGO IMPROCEDENTE a reclam...",não provido,MONOCRÁTICA,DIREITO DA SAÚDE | PÚBLICA | TRATAMENTO MÉDICO...,1 - DIREITO DA SAÚDE || PÚBLICA || TRATAMENTO ...,2023,02,2023.02
4,Rcl-57766,Rcl,57766,2023-02-01,MIN. EDSON FACHIN,Distribuido,VERA LUCIA GONCALVES DOS SANTOS,QUARTA TURMA RECURSAL DOS JUIZADOS ESPECIAIS D...,Violação,nan,...,2023-05-31,Negado seguimento,-,não provido,MONOCRÁTICA,DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...,1 - DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS D...,2023,02,2023.02


In [278]:
# Verificar se há alguma coluna duplicada
df_total.columns.duplicated().sum()

0

In [279]:
df_total['Indicador Usurpação'].value_counts()

Violação     2576
nan           218
Usurpação     124
violação       44
                3
Ursupação       1
violação        1
Name: Indicador Usurpação, dtype: int64

In [280]:
indicador_usurpacao = pd.DataFrame.from_dict({
    'Valor antigo': {0: 'Ursupação',	1: 'Usurpação',	2:'Violação',	3:'violação'},
    'Valor novo': {0: 'Usurpação',	1: 'Usurpação',	2:'Violação',	3:'Violação'}
                                            })

indicador_usurpacao

,Valor antigo,Valor novo
0,Ursupação,Usurpação
1,Usurpação,Usurpação
2,Violação,Violação
3,violação,Violação


In [281]:
# Substituir valores
df_total['Indicador Usurpação'] = df_total['Indicador Usurpação'].str.strip() # TRIM
df_total['Indicador Usurpação'] = df_total['Indicador Usurpação'].replace(['Ursupação'], 'Usurpação')
df_total['Indicador Usurpação'] = df_total['Indicador Usurpação'].replace(['violação'], 'Violação')
df_total['Indicador Usurpação'] = df_total['Indicador Usurpação'].replace('',np.nan,regex = True) # Replace blank by "NAN"

In [282]:
df_total['Indicador Usurpação'].value_counts()

Violação     2621
nan           218
Usurpação     125
Name: Indicador Usurpação, dtype: int64

In [283]:
# df_total = df_total.reset_index(drop=True)
# df_total.head()
# df = df.loc[:,~df.columns.duplicated()]
# df_total = df_total.loc[:,~df_total.columns.duplicated(keep='first')]

In [284]:
df_total['ano_mes_autu'].value_counts()

2023.03    698
2023.05    653
2023.04    622
2023.06    601
2023.02    393
Name: ano_mes_autu, dtype: int64

## Análise Bivariada

In [162]:
df_total.shape

(2967, 25)

In [285]:
#class limpar_base:

#    def __init__(self, df):
#        self.df = df

#    def processar_texto(self):
#        self.df = self.df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
#        self.df = self.df.applymap(lambda x: x.replace(' ', '_') if isinstance(x, str) else x)
        # df_obj = self.df.select_dtypes(['object'])
        #self.df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
        #self.df[df_obj.columns] = df_obj.apply(lambda x: x.replace('',np.nan,regex = True))
        #self.df[df_obj.columns] = df_obj.apply(lambda x: x.replace(['nan'], np.nan))
        #self.df[df_obj.columns] = df_obj.apply(lambda x: x.str.lower())

#        return self.df


#import pandas as pd

In [286]:
# Classe para limpar o texto

class limpar_base:
    import re
    import numpy as np

    def __init__(self, dataframe):
        self.dataframe = dataframe

    def processar(self):
        self.dataframe = self.dataframe.applymap(lambda x: x.strip() if isinstance(x, str) else x)
        self.dataframe = self.dataframe.applymap(lambda x: re.sub(r'\s+', ' ', x) if isinstance(x, str) else x)
        self.dataframe.replace('', np.nan, inplace = True)
        self.dataframe.replace('nan', np.nan, inplace = True)
        self.dataframe = self.dataframe.applymap(lambda x: x.lower() if isinstance(x, str) else x)

        return self.dataframe

# Example DataFrame
data = {
    'column1': ['  HELLO   world  ', '   python   programming', 'data science   ', 'nan'],
    'column2': ['  Data analysis  ', '   Machine learning', 'deep learning   ', '']
}

df = pd.DataFrame(data)

# Create an instance of DataFrameProcessor
processor = limpar_base(df)

# Process the entire DataFrame and get the modified DataFrame
modified_df = processor.processar()

print(modified_df)

              column1           column2
0         hello world     data analysis
1  python programming  machine learning
2        data science     deep learning
3                 NaN               NaN


In [290]:
# Create an instance of DataFrameProcessor
processor = limpar_base(df_total)

# Process the entire DataFrame and get the modified DataFrame
df_total = processor.processar()

In [292]:
df_total['Indicador Usurpação'].value_counts()

violação     2621
usurpação     125
Name: Indicador Usurpação, dtype: int64

In [293]:
# Vamos retirar os dados faltantes
df_total.dropna(subset = ['Indicador Usurpação'], inplace = True)

In [295]:
# Andamento por mês

pd.crosstab(df_total['Andamento decisão'],
            df_total['ano_mes_autu'],
            values = df_total['Link Processo'],
            aggfunc='count')

ano_mes_autu,2023.02,2023.03,2023.04,2023.05,2023.06
Andamento decisão,,,,,
concedida a ordem de ofício,1.0,NaN,1.0,NaN,1.0
decisão (segredo de justiça),1.0,2.0,1.0,5.0,NaN
decisão referendada,NaN,2.0,NaN,1.0,NaN
deferido,NaN,2.0,NaN,NaN,NaN
deferido em parte,1.0,NaN,NaN,NaN,NaN
extinto o processo,8.0,12.0,10.0,14.0,6.0
homologada a desistência,3.0,9.0,6.0,10.0,7.0
improcedente,12.0,16.0,12.0,15.0,18.0
liminar deferida,1.0,5.0,NaN,1.0,3.0


In [296]:
pd.crosstab(df_total['Andamento decisão'],
            df_total['Indicador Usurpação'],
            values = df_total['Link Processo'],
            aggfunc='count')


Indicador Usurpação,usurpação,violação
Andamento decisão,,
concedida a ordem de ofício,NaN,3.0
decisão (segredo de justiça),1.0,8.0
decisão referendada,NaN,3.0
deferido,NaN,2.0
deferido em parte,NaN,1.0
extinto o processo,2.0,48.0
homologada a desistência,2.0,33.0
improcedente,5.0,68.0
liminar deferida,NaN,10.0


In [297]:
# Quantidade de dados faltantes em cada uma das variáveis
pd.DataFrame.from_dict({
    'Variável': {0: 'Indicador Usurpação',
                 1: 'Andamento decisão',
                 2:'-',
                 3:'-'},
    'Qtd. NaN': {0: df_total['Indicador Usurpação'].isna().sum(),
                 1: df_total['Andamento decisão'].isna().sum(),
                 2:'-',
                 3:'-'}
                                            })

#(df_total['Indicador Usurpação'].isna().sum(), df_total['Andamento decisão'].isna().sum())

,Variável,Qtd. NaN
0,Indicador Usurpação,0
1,Andamento decisão,410
2,-,-
3,-,-


In [65]:
# Processos SEM classificação de "Indicador de Usurpação"
# df_total.loc[df_total['Indicador Usurpação'].isnull()].head()

In [298]:
# Tabela de provimento em RCL por Usurpacao ou Competência
pd.crosstab(df_total['Provimento'], df_total['Indicador Usurpação'], values = df_total['Link Processo'], aggfunc='count')

Indicador Usurpação,usurpação,violação
Provimento,,
não provido,99,1777
provido,26,844


In [299]:
# Tabela de provimento em RCL por relator
tab_prov_relator = pd.crosstab(df_total['Relator Atual'],
                               df_total['Provimento'],
                               values = df_total['Link Processo'],
                               aggfunc='count').round(0)

# tab_prov_relator['total'] = tab_prov_relator['total'].apply(lambda x: x/x.sum(), axis=1).round(2)

#tab_prov_relator[['% nao provido', '% provido']] = tab_prov_relator[['não provido', 'provido']].apply(lambda x: x/x.sum(), axis=1).round(2)

tab_prov_relator

Provimento,não provido,provido
Relator Atual,,
min. alexandre de moraes,177.0,184.0
min. andré mendonça,218.0,73.0
min. cármen lúcia,179.0,113.0
min. dias toffoli,203.0,97.0
min. edson fachin,261.0,32.0
min. gilmar mendes,155.0,119.0
min. luiz fux,216.0,89.0
min. nunes marques,245.0,47.0
min. ricardo lewandowski,37.0,9.0


In [301]:
# Tabela de provimento em RCL por relator
tab_prov_relator = pd.crosstab(df_total['Relator Atual'],
                               df_total['Indicador Usurpação'],
                               values = df_total['Link Processo'],
                               aggfunc='count').round(0).fillna(0)

tab_prov_relator

Indicador Usurpação,usurpação,violação
Relator Atual,,
min. alexandre de moraes,22,339
min. andré mendonça,10,281
min. cármen lúcia,12,280
min. dias toffoli,10,290
min. edson fachin,14,279
min. gilmar mendes,14,260
min. luiz fux,11,294
min. nunes marques,13,279
min. ricardo lewandowski,3,43


In [304]:
# Analise de provimento pela violação - USURPACAO

filter_usurpacao = df_total[df_total['Indicador Usurpação'] == 'usurpação']
# Tabela de provimento em RCL por relator
filter_usurpacao = pd.crosstab(filter_usurpacao['Relator Atual'],
                               filter_usurpacao['Provimento'],
                               values = filter_usurpacao['Link Processo'],
                               aggfunc='count').round(0).fillna(0)

filter_usurpacao

Provimento,não provido,provido
Relator Atual,,
min. alexandre de moraes,16.0,6.0
min. andré mendonça,7.0,3.0
min. cármen lúcia,9.0,3.0
min. dias toffoli,10.0,0.0
min. edson fachin,12.0,2.0
min. gilmar mendes,11.0,3.0
min. luiz fux,7.0,4.0
min. nunes marques,10.0,3.0
min. ricardo lewandowski,1.0,2.0


In [305]:
# Analise de provimento pela violação - VIOLACAO - GERAL

filter_violacao = df_total[df_total['Indicador Usurpação'] == 'violação']
# Tabela de provimento em RCL por relator
filter_violacao = pd.crosstab(filter_violacao['Relator Atual'],
                               filter_violacao['Provimento'],
                               values = filter_violacao['Link Processo'],
                               aggfunc='count').round(0).fillna(0)

filter_violacao

Provimento,não provido,provido
Relator Atual,,
min. alexandre de moraes,161.0,178.0
min. andré mendonça,211.0,70.0
min. cármen lúcia,170.0,110.0
min. dias toffoli,193.0,97.0
min. edson fachin,249.0,30.0
min. gilmar mendes,144.0,116.0
min. luiz fux,209.0,85.0
min. nunes marques,235.0,44.0
min. ricardo lewandowski,36.0,7.0


In [337]:
# limpeza na coluna de tema
df_total_2 = df_total
# df_teste = df_total['Repercussão Geral'].replace(' e ', ';')

# 1
# O que não houver a palavra tema será apagado
df_total_2.loc[~df_total_2['Repercussão Geral'].str.contains('tema', na=False), 'Repercussão Geral'] = ''

In [343]:
# Substituir algumas palavras chaves

df_total_2['Repercussão Geral'].replace(' e ', ';', inplace = True)
df_total_2['Repercussão Geral'].replace(',tema', '; tema', inplace = True)
df_total_2['Repercussão Geral'].replace(', tema', '; tema', inplace = True)
df_total_2['Repercussão Geral'].replace('//', ';', inplace = True)
df_total_2['Repercussão Geral'].replace('\\', ';', inplace = True)

In [344]:
rg_count = df_total_2['Repercussão Geral'].value_counts()
rg_count.to_excel('rg.xlsx', index=True)

In [306]:
# Analise de provimento pela violação - VIOLACAO - REPERCUSSÃO GERAL

filter_violacao = df_total[df_total['Indicador Usurpação'] == 'violação']
# Tabela de provimento em RCL por relator
filter_violacao = pd.crosstab(filter_violacao['Repercussão Geral'],
                              filter_violacao['ano_mes_autu'],
                              values = filter_violacao['Link Processo'],
                              aggfunc='count').round(0).fillna(0).sort_values(by='2023.06', ascending = False).iloc[0:10,]

filter_violacao

ano_mes_autu,2023.02,2023.03,2023.04,2023.05,2023.06
Repercussão Geral,,,,,
tema 246,29.0,46.0,96.0,51.0,55.0
tema 1170; 810,0.0,0.0,0.0,0.0,22.0
tema 725,11.0,11.0,16.0,28.0,20.0
tema 1232,0.0,2.0,0.0,0.0,13.0
tema 339,4.0,9.0,0.0,5.0,8.0
tema 1046,12.0,10.0,22.0,3.0,7.0
tema 793,9.0,16.0,9.0,9.0,6.0
tema 1199,4.0,0.0,0.0,1.0,4.0
temas 928 e 136,0.0,0.0,0.0,0.0,4.0


In [308]:
# Analise de provimento pela violação - VIOLACAO - CONTROLE CONCENTRADO.

filter_violacao = df_total[df_total['Indicador Usurpação'] == 'violação']
# Tabela de provimento em RCL por relator
filter_violacao = pd.crosstab(filter_violacao['Controle Concentrado'],
                              filter_violacao['ano_mes_autu'],
                              values = filter_violacao['Link Processo'],
                              aggfunc='count').round(0).fillna(0).sort_values(by='2023.06', ascending = False).iloc[0:10,]

filter_violacao

ano_mes_autu,2023.02,2023.03,2023.04,2023.05,2023.06
Controle Concentrado,,,,,
adc 16,26.0,48.0,92.0,17.0,40.0
adpf 324,17.0,12.0,16.0,24.0,14.0
adi 3395/df,0.0,0.0,0.0,1.0,14.0
adpf 828,9.0,18.0,14.0,6.0,11.0
adc nº 16/df,0.0,0.0,0.0,16.0,10.0
adc 48,8.0,7.0,18.0,6.0,10.0
adpf 528,0.0,0.0,0.0,3.0,9.0
adc nº 48,0.0,0.0,0.0,9.0,8.0
adi 3395,3.0,16.0,10.0,3.0,7.0


In [310]:
# Analise de provimento pela violação - VIOLACAO

filter_violacao = df_total[df_total['Indicador Usurpação'] == 'violação']
# Tabela de provimento em RCL por relator
filter_violacao = pd.crosstab(filter_violacao['Controle Difuso'],
                              filter_violacao['ano_mes_autu'],
                              values = filter_violacao['Link Processo'],
                              aggfunc='count').round(0).fillna(0).sort_values(by='2023.06', ascending = False).iloc[0:10,]

filter_violacao

ano_mes_autu,2023.02,2023.03,2023.04,2023.05,2023.06
Controle Difuso,,,,,
re 760.931/df,0.0,0.0,0.0,1.0,9.0
re 958.252,1.0,1.0,0.0,1.0,7.0
rhc nº 194.586/pr,0.0,0.0,0.0,0.0,1.0
re 583.9559/rj,0.0,0.0,0.0,0.0,1.0
hc 69.013,0.0,0.0,0.0,0.0,1.0
hc n° 218.287/rj,0.0,0.0,0.0,0.0,1.0
hc n° 824059,0.0,0.0,0.0,0.0,1.0
hc nº 227.328,0.0,0.0,0.0,0.0,1.0
re 1387795/mg,0.0,0.0,0.0,0.0,1.0


In [311]:
# Tabela de provimento em RCL por relator
tab_prov_relator = pd.crosstab(df_total['ano_mes_autu'],
                               df_total['Indicador Usurpação'],
                               values = df_total['Link Processo'],
                               aggfunc='count').round(0).fillna(0)

tab_prov_relator

Indicador Usurpação,usurpação,violação
ano_mes_autu,,
2023.02,14,364
2023.03,46,463
2023.04,26,584
2023.05,23,630
2023.06,16,580
